Requirement

1.Folder
     * './mart' 폴더에 (1)에서 저장한 Pickle 데이터가 들어 있어야 함
     * './saved_models' 폴더에 (2)에서 저장한 Keras Model 데이터가 들어 있어야 함
     * './ckpt' 폴더가 생성되어 있어야 함
2.Package
     * Python3.6
     * pandas, numpy, os, time, pickle, tensorflow, keras, matplotlib, sklearn

## Data 불러오기
* (1)에서 저장한 데이터 중, (X, y)를 불러온다.
* Model의 Generalization 성능을 측정하기 위해 Train / Validation data split을 0.85 : 0.15 비율로 나눈다.
* preproc_for_seq 함수를 통해, Sequence data를 학습하기 위한 전처리를 진행한다.

In [ ]:
import pandas as pd 
import numpy as np
import os, time, pickle
import tensorflow as tf
import keras
from keras.layers.recurrent import GRU
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *

def write_pickle(data, path, file_name):
    with open("".join([path, '/', file_name, '.pkl']), 'wb') as f:
        pickle.dump(data, f)


def read_pickle(path, file_name):
    with open("".join([path, '/', file_name, '.pkl']), 'rb') as f:
        return pickle.load(f)

mart_path = './mart'
time_length = 44
timelength = time_length

In [ ]:
X, y = read_pickle(mart_path, '44_develop_data')
feature_cols = read_pickle(mart_path, '44_feature_cols')

In [ ]:
seq_cols = feature_cols[:time_length]
print(seq_cols)

meta_cols = feature_cols[time_length:]
print(meta_cols)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
def preproc_for_seq(array):
    
    array_seq = []
    
    # seq_cols
    array_seq.append(array[:,:timelength].reshape((array.shape[0], timelength, 1)))
    
    # meta_cols
    array_seq.append(array[:,timelength:])
    
    return array_seq

X_seq = preproc_for_seq(X)
y_seq = y

X_train_seq = preproc_for_seq(X_train)
X_valid_seq = preproc_for_seq(X_valid)

## Predictive Modeling - ResNet & DNN FineTuning

### Encoder 불러오기
* (2)에서 저장했던 Encoder를 불러온다

In [ ]:
def save_keras_model(model, filename):
    model_json = model.to_json()
    with open('{}.json'.format(filename), 'w') as json_file:
        json_file.write(model_json)
    model.save_weights('{}.h5'.format(filename))

def load_keras_model(filename):
    from keras.models import model_from_json
    json_file = open('{}.json'.format(filename), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights('{}.h5'.format(filename))
    return model

In [ ]:
model_dir = './saved_models'
filename = '{}/ResNetAE_encoder'.format(model_dir)
encoder = load_keras_model(filename)

### Define Architecture
* Encoder의 Output 위에 Dense Layer를 하나만 쌓는다.

In [ ]:
seq_in, meta_in = encoder.inputs
merges = encoder.outputs[0]
y = Dense(1, kernel_initializer='he_normal', name='output')(merges)

model = Model(inputs=[seq_in, meta_in], outputs=y)

In [ ]:
model.summary()

### Define Loss & Optimizer
* Loss는 Competition 주최측에서 공지한 Evaluation Function * (-1)을 사용한다.
* custom_loss가 그 역할을 수행한다.
* optimizer는 Adam을 사용하고 Learning Rate는 0.001이다.

In [ ]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    default_position = tf.less(y_true, y_pred)
    profit_position = tf.logical_not(default_position)
    pos = tf.reduce_sum(y_pred*tf.cast(profit_position, tf.float32)*13/365)
    neg = tf.reduce_sum((y_true-y_pred)*tf.cast(default_position, tf.float32))
    loss = -pos-neg
    return loss

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss=custom_loss, optimizer=optimizer)

### Define Callbacks
* Learning Rate를 잘 조절해준다는 Cyclic lr을 사용 (LRFinder Class)
* epoch = 100, batch_size = 128
* ModelCheckpoint Callback을 사용해서, Validation data에서 역대 최저 Loss를 갱신할 때마다, 모델을 저장

In [ ]:
%matplotlib inline
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import keras.backend as K

class LRFinder(Callback):
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')

In [ ]:
epochs = 100
batch_size = 128
epoch_size = len(X_train)

lr_finder = LRFinder(min_lr=1e-5, 
                     max_lr=1e-3, 
                     steps_per_epoch=np.ceil(epoch_size/batch_size), 
                     epochs=epochs)

early_stop = keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')

ckpt_dir = './ckpt'
ckpt_path = ckpt_dir + '/ResNetFinetuning_{epoch:02d}_valloss{val_loss:.2f}.hdf5'
ckpt = keras.callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

#callbacks = [lr_finder]

### Model Training

In [ ]:
history = model.fit(X_train_seq, y_train,
                    batch_size=batch_size, epochs=epochs,
                    callbacks=[ckpt, lr_finder],
                    validation_data=(X_valid_seq, y_valid), shuffle=True)

### Best Model Loading
* 위 Training Log를 봤을 때, 국소적 역대 최고 Validation Loss를 기록한 것은, epoch 37, epoch 76, epoch 91, epoch 97 네 개이다
* 아주 역대 최고는 epoch 97이지만, 76 ~ 91, 36 ~76 사이에 Validation Loss 하락 공백이 두 번 있었기 때문에, Overfitting을 의심하지 않을 수 없다.
* 결론적으로, Competition에 Submit을 했을 때, 아래와 같은 Competition 점수를 얻었다
    * epoch 37 = 53.8752점
    * epoch 76 = 52.48691점 (3등 달성)
    * epoch 91 = 66.65879점
    * epoch 97 = 92점
* 따라서, epoch 76에서의 ResNet + DNN Finetuning 모델은 training Loss 1.15, validation Loss 1.07, Test Score 52.48691의 우수한 성적을 내는 Overfitting이 되지 않으면서 우수한 Generalization 성능을 내는 우수한 단일 모델이라고 볼 수 있다.

In [ ]:
model.load_weights(ckpt_dir +'/ResNetFinetuning_76_valloss1.07.hdf5')

In [ ]:
def save_keras_model(model, filename):
    model_json = model.to_json()
    with open('{}.json'.format(filename), 'w') as json_file:
        json_file.write(model_json)
    model.save_weights('{}.h5'.format(filename))

def load_keras_model(filename):
    from keras.models import model_from_json
    json_file = open('{}.json'.format(filename), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights('{}.h5'.format(filename))
    return model

In [ ]:
model_dir = './saved_models'
filename = '{}/ResNetFinetuning'.format(model_dir)

save_keras_model(model, filename)

## Prediction

### Test Data, Validation Data에 대해 y를 Predict한다.
* Test Data: Submit을 위해
* Validation Data: Stacking 등을 위해 (하지 않았지만)
* y는 Scaling이 되어 있는 상태이므로, inverse_transform을 수행한다

In [ ]:
X_test = read_pickle(mart_path, '{}_X_test'.format(timelength))
y_scaler = read_pickle(mart_path, '{}_y_scaler'.format(timelength))

X_test_seq = preproc_for_seq(X_test)

y_pred = model.predict(X_test_seq)
y_pred_valid = model.predict(X_valid_seq)

y_pred = y_scaler.inverse_transform(y_pred)
y_pred_valid = y_scaler.inverse_transform(y_pred_valid)
y_true_valid = y_scaler.inverse_transform(y_valid)

## Test Data에 대한 Prediction을 Submission 양식에 맞추어 만들어낸다.
* (1) Wranling 파트에서 store_id를 중복을 피하기 위해 조작을 했으므로, 본래의 store_id를 복구하기 위해 test_data를 다시 불러와서 원래의 store_id 정보를 얻어낸다.
* Test data에 대한 Prediction과 함께 조합하여, Submission 파일을 만든다.

In [ ]:
# Data Meta 정보 지정
data_path = "./data/"
train_filename = "train.csv"
test_filename = "test.csv"

train_data = pd.read_csv(data_path + train_filename)
test_data = pd.read_csv(data_path + test_filename)


In [ ]:
submission = pd.DataFrame({'store_id': test_data['store_id'].unique(), 'total_sales': y_pred.reshape(len(y_pred))})
submission.to_csv('submission_ResNetFinetuning76epoch.csv', index=False)

### Validation Data에 대한 Prediction을 추후 Stacking / Blending 등을 위해 저장한다

In [ ]:
write_pickle(y_pred_valid, ".", 'y_pred_valid_ResNetFinetuning76epoch')
write_pickle(y_true_valid, ".", 'y_true_valid_ResNetFinetuning76epoch')

## Evaluation
* 대회 주최측에서 알려준 Evaluation metric을 Python Function으로 만들었다.
* Validation에 Evaluation Function을 적용했을 때, 101732063을 얻었고, 이는 만점 대비 0.4692700349674595%의 성과이다.
* 이는 내가 지금까지 Evaluation했을 때 결과 중 Top 5 안에 들면서, Submit했을 때도 높은 점수를 얻은 결과라 할 수 있다.

In [ ]:
def get_score(y_true, y_pred, verbose=True):
    
    assert len(y_true) == len(y_pred)
    
    y_true = y_true.reshape(len(y_true),)
    y_pred = y_pred.reshape(len(y_true),)
    
    default_position = y_true - y_pred &lt; 0
    profit_position = ~default_position
    pos = np.sum(y_pred[profit_position]*13/365)
    neg = np.sum(y_true[default_position] - y_pred[default_position])
    score = pos + neg
    if verbose:
        print("The positive score: ")
        print(pos)
        print("The negative score: ")
        print(neg)
        print("The total score: ")
        print(score)
    return score

In [ ]:
score = get_score(y_true_valid, y_pred_valid)

In [ ]:
def get_percentage_to_upperbd(y_true, y_pred, verbose=False):
    upperbd_score = get_score(y_true, y_true, verbose=False)
    score = get_score(y_true, y_pred, verbose=False)
    percentage = score/upperbd_score
    if verbose:
        print("The upper bound score: ")
        print(upperbd_score)
        print("The obtained score: ")
        print(score)
        print("The percentage to upper bound: ")
        print(percentage)
    return percentage

In [ ]:
percentage = get_percentage_to_upperbd(y_true_valid, y_pred_valid, verbose=True)